### Engineering

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
lazy loading
memory mapping
data prefetching

In [ ]:
#include <iostream>

In [ ]:
for (; i <= 9; i++) {
    std::cout << "i = " << i << std::endl;
}

In [ ]:
typedef int age_type;

In [ ]:
int* ptr = (int*)malloc(69)

In [ ]:
for (int x: xs) {
    std::cout << "value: " << x << std::endl;
}

In [ ]:
def compute_forward_pass_using_data_parallelism(
    model, input,
    decide_ids, output_id
):
    models = nn.parallel.replicate(model, device_ids)
    inputs = nn.parallel.scatter(input, device_ids)
    
    logit = nn.parallel.parallel_apply(models, inputs)
    logits = nn.parallel.gather(logit, output_id)
    
    return logits

In [6]:
class f(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return input.clone()
    
    @staticmethod
    def backward(ctx, grad_input):
        torch.distributed.all_reduce(grad_input)
        return grad_input

In [5]:
class g(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        world_size = torch.distributed.get_world_size()
        inputs = [torch.empty_like(input) for _ in range(world_size)]
        torch.distributed.all_gather(inputs, input)
        inputs = torch.cat(inputs, dim=-1)
        return inputs
    
    @staticmethod
    def backward(ctx, grad_input):
        rank = torch.distributed.get_rank()
        world_size = torch.distributed.get_world_size()
        dim_size = grad_input.shape[-1]
        dim_size_per_partrition = dim_size // world_size
        grad_chunks = torch.split(grad_input, dim_size_per_partrition, dim=-1)
        return grad_chunks[rank]

In [4]:
class ColumnParallelLinear(nn.Module):
    def __init__(self, input_size, output_size, world_size):
        super().__init__()
        self.input_size = input_size
        self.output_size_per_partrition = output_size // world_size
        self.weight = nn.Parameter(torch.empty(
            self.output_size_per_partrition,
            self.input_size
        ))
        self.bias = nn.Parameter(torch.empty(
            self.output_size_per_partrition
        ))
    
    def forward(self, input):
        input_parallel = f.apply(input)
        output_parallel = F.linear(
            input_parallel,
            self.weight,
            self.bias
        )
        output = g.apply(output_parallel)
        return output

In [ ]:
int* h_a, h_b, h_c;

In [ ]:
size_t bytes = sizeof(int)*n;

In [ ]:
h_a = (int*)malloc(bytes)
h_b = (int*)malloc(bytes)
h_c = (int*)malloc(bytes)

In [7]:
import functools

In [8]:
def my_decorator(func):
    @functools.wraps(func)
    def wrapper():
        print("before")
        func()
        print("fater")
    
    return wrapper

In [9]:
def by_row_parallelism(inputs, weights):
    inp_dim_partrition = inputs.shape[-1] // 2
    w_dim_partrition = weights.shape[0] // 2
    
    inp1 = inputs[:, :inp_dim_partrition]
    inp2 = inputs[:, inp_dim_partrition:]
    
    w1 = weights[:w_dim_partrition, :]
    w2 = weights[w_dim_partrition:, :]
    
    out1 = inp1 @ w1
    out2 = inp2 @ w2
    
    out = out1 + out2
    
    return out

In [15]:
class f(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return input.clone()
    
    @staticmethod
    def backward(ctx, grad_input):
        torch.distributed.all_reduce(grad_input)
        return grad_input

In [14]:
class g(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        world_size = torch.distributed.get_world_size()
        inputs = [torch.empty_like(input) for _ in range(world_size)]
        torch.distributed.all_gather(inputs, input)
        inputs = torch.cat(inputs, dim=-1)
        return inputs
    
    @staticmethod
    def backward(ctx, grad_input):
        rank = torch.distributed.get_rank()
        world_size = torch.distributed.get_world_size()
        dim_size = grad_input.shape[-1]
        dim_size_per_partrition = dim_size // world_size
        grad_chunks = torch.split(grad_input, dim_size_per_partrition, dim=-1)
        return grad_chunks[rank]

In [11]:
class ColumnParallelLinear(nn.Module):
    def __init__(self, input_size, output_size, world_size):
        super().__init__()
        self.input_size = input_size
        self.output_size_per_partrition = output_size // world_size
        
        self.weight = nn.Parameter(torch.empty(
            self.output_size_per_partrition,
            self.input_size
        ))
        self.bias = nn.Parameter(torch.empty(
            self.output_size_per_partrition
        ))
    
    def forward(self, input):
        input_parallel = f.apply(input)
        output_parallel = F.linear(
            input_parallel,
            self.weight,
            self.bias
        )
        outputs = g.apply(output_parallel)
        return outputs

In [ ]:
def by_column_parallelism(inputs, weights):
    dim_size = weights.shape[-1]
    dim_size_per_partrition = dim_size // 2
    
    w1 = weights[:, :dim_size_per_partrition]
    w2 = weights[:, dim_size_per_partrition:]
    
    out1 = inputs @ w1
    out2 = inputs @ w2
    
    return torch.cat([out1, out2], dim=-1)

In [16]:
from torch.utils.data import Dataset

In [17]:
class CachedDataset(Dataset):
    def __init__(self, filename):
        super().__init__()
        self.filename = filename
        self.data = None
        self.cached_idxs = []
        self.cache = None
    
    def prefetch(self, idxs):
        if all([i in self.cached_idxs for i in idxs]):
            return
        
        if not self.data:
            self.data = torch.load(self.filename)
        
        total_elements = sum([self.data[i].numel() for i in idxs])
        self.cache = torch.zeros(total_elements, dtype=self.data.dtype)
        self.cached_idxs.clear()
        
        offset = 0
        for i in idxs:
            n_elements = self.data[i].numel()
            self.cache[offset:offset+n_elements] = self.data[i].view(-1)
            self.cached_idxs.append(i)
            offset += n_elements

In [ ]:
forward and backward pass, model parameters, optimizer

In [ ]:
from t

In [18]:
import evidently

In [22]:
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

In [23]:
report = Report(metrics=[DataDriftPreset()])

In [ ]:
report.run(reference_data=reference_data, current_data=current)

In [24]:
from airflow.decorators import dag
from airflow.operators.python import PythonOperator

In [25]:
def task_1(ti):
    ti.push("x", 1)

In [26]:
def task_2(ti):
    x = ti.get("x")
    y = x + 1
    ti.push("y", y)

In [ ]:
@dag(dag_id=dag_id, start_date=start_date)
def workflow():
    task_1 = PythonOperator(python_callable=task_1, task_id="task_1")
    task_2 = PythonOperator(python_callable=task_2, task_id="task_2")
    
    task_1 >> task_2

In [ ]:
import pyspark.sql.functions as F

In [ ]:
new_df = df.withColumn("date", F.to_date(df.date))

In [28]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
torch.roll(x, shifs=1, dim=1)

In [ ]:
handles = [model.ln_final.register_forward_pre_hook(hook) for hook in hooks]

In [ ]:
handles[1].remove()

In [29]:
from transformer_lens import HookedTransformerConfig, HookedTransformer

In [ ]:
cfg = HookedTransformerConfig(**params)
model = HookedTransformer(cfg=cfg)

In [ ]:
step 1: two prompt
step 2: choose a component
step 3: record all the interdimate activations
step 4:

In [30]:
from contextlib import contextmanager

In [ ]:
@contextmanager
def use_hooks(model, hooks):
    try:
        module = model.blocks[1]
        handles = [module.register_forward_pre_hook(hook) for hook in hooks]
    finally:
        for handle in handles:
            handle.remove()

In [ ]:
_, cache = model.run_with_cache(tokens)

In [31]:
from transformer_lens.utils import get_act_name

In [32]:
hook_name = get_act_name("pattern", 0, "attn")

In [33]:
hook_name

'blocks.0.attn.hook_pattern'

In [ ]:
attention_pattern = cache[hook_name]

In [34]:
import circuitsvis as cv

In [ ]:
str_tokens = model.to_str_tokens(tokens)

In [ ]:
cv.attention.attention_pattern(
    tokens=str_tokens,
    attention=attention_pattern
)

In [ ]:
def print_shape(module, input):
    print(input.shape)

In [ ]:
model.blocks[1].register_forward_pre_hook(print_shape)

In [36]:
from evidently.metric_preset import DataDriftPreset
from evidently.report import Report

In [37]:
report = Report(
    metrics=[DataDriftPreset()]
)

In [ ]:
report.run(reference_data)

In [ ]:
W_U = model.W_U

In [ ]:
tokens = model.to_tokens(text)
tokens = tokens[0]

In [ ]:
target_tokens = tokens[1:]

In [ ]:
W_U_correct_tokens = W_U[:, target_tokens]

In [38]:
import torch.distributed.rpc as rpc

In [ ]:
rpc.rpc_sync()

In [ ]:
tokens = model.to_tokens(repeated_text)

In [ ]:
induction_heads = [(6, 9), (4, 2)]

In [39]:
attention_patterns = []

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
for head_idx, layer_idx in induction_heads:
    hook_name = get_act_name("attn", layer_idx)
    attention_patterns.append(
        cache[hook_name][0, head_idx]
    )

In [41]:
import os
import torch.multiprocessing as mp

In [42]:
AGENT_NAME = "agent_{}"

In [ ]:
def run_worker(rank, world_size):
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "29500"
    
    torch.distributed.rpc.init_rpc(
        AGENT_NAME.format(rank)
        rank=rank,
        world_size=world_size
    )

In [ ]:
for rank in range(world_size):
    process = mp.Process(
        target=run_worker,
        args=(rank, world_size)
    )

In [ ]:
agent_rref.rpc_sync().init("hello")

In [43]:
from torch.distributed.rpc import RRef

In [44]:
class Agent:
    def __init__(self):
        self.id = RRef(self)

In [45]:
futs = []

In [ ]:
for ob_rref in ob_rrefs:
    futs.append(ob_rref.rpc_async().run())

In [ ]:
for fut in futs:
    fut.wait()

In [ ]:
class Observer:
    def __init__(self, env):
        self.id = rpc.get_worker_info().id
        self.env = env
    
    def run_episode(self, agent_rref):
        state, _ = self.env.reset()
        
        for _ in range(69):
            action = agent_rref.rpc_sync().select_action(self.id, state)
            state, reward, done, _ = env.step(action)
            
            if done: break

In [ ]:
rpc.get_worker_info(worker_id).name 

In [46]:
futs = []

In [ ]:
for ob_rref in ob_rrefs:
    futs.append(rpc.async)

In [47]:
class LayerNorm(nn.Module):
    def __init__(self, features, eps):
        super().__init__()
        self.adds = nn.Parameter(torch.zeros(features))
        self.mults = nn.Parameter(torch.ones(features))
        self.eps = eps
    
    def forward(self, x):
        mean, var = x.mean(), x.var()
        x = (x-mean)/(self.eps+var).sqrt()
        x = self.mults*x + self.adds
        return x

In [ ]:
image_similarities = image_embeddings @ image_embeddings.T

In [ ]:
text_similarities = text_embeddings @ text_embeddings.T

In [ ]:
target = F.softmax(
    (image_similarities + text_similarities) / (2*temperature)
)

In [48]:
AGENT_NAME = "agent_{}"

In [ ]:
def run_worker(rank, world_size):
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "29500"
    
    rpc.init_rpc(
        AGENT_NAME.format(rank),
        rank=rank,
        world_size=world_size
    )

In [ ]:
for rank in range(world_size):
    process = mp.Process(
        target=run_worker,
        args=(rank, world_size)
    )
    process.start()

In [49]:
futs = []

In [ ]:
for ob_rref in ob_rrefs:
    futs.append(rpc.rpc_async(
        to=ob_rref.owner(),
        func=ob_rref.rpc_sync().run
    ))

In [ ]:
for fut in futs:
    fut.wait()